In [1]:
import requests
import base64
import pandas as pd
import uuid

In [2]:
user_phone_number = '9664608973'
# user_phone_number = '7900062448'
start_date = "2023-08-01"
end_date = "2023-12-01"

In [3]:
# Details from Perfios
BASE_URL = "https://fiu.perfios.com/fiu"
API_KEY = "db2113a28c175e01e03e847aada0ee8e376b4d630366cd73a2a6d30e3c12652b"
ORG_CODE = "Finalyca_FIU001"
PROFILE_ID = "finalyca_001"

# Random string to manage resources between multiple requests
TXN_ID = uuid.uuid4().hex
user_id = F"{user_phone_number}@anumati"
USER_ID = base64.b64encode(user_id.encode("ascii")).decode("ascii")

process_dict = dict()
process_dict["txn_id"] = TXN_ID

In [4]:
# Get the list of all FIP entities
def get_fip_info(api_key, org_code):
    headers = dict()
    headers["api_key"] = api_key
    headers["org_id"] = org_code
    resp = requests.get(F"{BASE_URL}/entity/FIP", headers=headers)
    return resp.json()

resp = get_fip_info(API_KEY, ORG_CODE)
df = pd.DataFrame(resp)
l = df["name"].unique()
l.sort()

In [5]:
status, redirect_url, txn_id, error_code = initiate_consent(BASE_URL, API_KEY, ORG_CODE, PROFILE_ID, USER_ID, TXN_ID, start_date, end_date)
process_dict["consent_initiate_time"] = int(time.time())
process_dict["consent_status"] = status

In [6]:
redirect_url

'https://fiu.perfios.com/fiu/v1/api/web/Consent/Request/28be3f6e-61c8-45f8-8b2b-320393b4cb47'

In [7]:
status, accounts, error_code = check_transaction_status(BASE_URL, API_KEY, ORG_CODE, TXN_ID)
print(status, error_code)

final_accounts = list()
if accounts:
    process_dict["consent_validate_time"] = int(time.time())

    for account in accounts:
        obj = dict()
        obj["fipId"] = account["fipId"]
        obj["linkRefNumber"] = account["linkRefNumber"]
        obj["fiType"] = account["fiType"]
        final_accounts.append(obj)

COMPLETED None


In [8]:
statements = list()
for acc in final_accounts:
    fip_id = acc["fipId"]
    link_ref_number = acc["linkRefNumber"]

    process_dict["report_fetch_time"] = int(time.time())
    resp = get_report(BASE_URL, API_KEY, ORG_CODE, TXN_ID, fip_id, link_ref_number)
    process_dict["report_receive_time"] = int(time.time())

    d = resp.json()["data"]
    xml_str = base64.b64decode(d).decode('utf-8')

    statements.append(process_accounts(xml_str))

In [9]:
pd.DataFrame(statements[0]["holdings"])

,amc,amfiCode,closingUnits,FatcaStatus,folioNo,isin,isinDescription,lienUnits,lockinUnits,nav,navDate,registrar,schemeCategory,schemeCode,schemeOption,schemeTypes,ucc
0,4,101844,0,Compliant,1014810495,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,38.8716,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,129,GROWTH_TYPE,DEBT_SCHEMES,
1,4,102767,0,Compliant,1014810495,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,37.511,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,130,GROWTH_TYPE,DEBT_SCHEMES,
2,4,108273,7249.847,Compliant,1040100595,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,299.8947,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,171,GROWTH_TYPE,DEBT_SCHEMES,
3,7,102448,0,Compliant,298709/56,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,44.4523,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,261,GROWTH_TYPE,DEBT_SCHEMES,
4,7,101304,66679.731,Compliant,298709/56,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,38.327,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,271,GROWTH_TYPE,DEBT_SCHEMES,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,10,148001,0,Compliant,17438774,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,34.9077,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,40204,GROWTH_TYPE,DEBT_SCHEMES,
138,10,148318,771.017,Compliant,17438774,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,0.5036,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,40324,GROWTH_TYPE,DEBT_SCHEMES,
139,23,148094,0,Compliant,41430843692,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,99.2,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,40365,GROWTH_TYPE,DEBT_SCHEMES,
140,23,148268,0,Compliant,499189046566,IN000AAA000A,ABC BANK LIMITED EQ LISTING,0,0,302.77,2022-02-04,Either CAMS / KARVY,LARGE_CAP_FUND,40369,GROWTH_TYPE,DEBT_SCHEMES,


In [10]:
pd.DataFrame(statements[1]["holdings"])

,isin,isinDescription,issuerName,lastTradedPrice,units
0,INE748C01038,3I Infotech Ltd.,3I Infotech Ltd.,47.3,1400
1,INE030A01027,Hindustan Unilever Ltd.,Hindustan Unilever Ltd.,2611.0,50
2,INE154A01025,ITC Ltd.,ITC Ltd.,326.6,2000
3,INE663F01024,Info Edge India Ltd.,Info Edge India Ltd.,4261.75,10
4,INE101A01026,Mahindra & Mahindra Ltd.,Mahindra & Mahindra Ltd.,1308.75,250
5,INE338I01027,Motilal Oswal Financial Services Ltd.,Motilal Oswal Financial Services Ltd.,780.0,300
6,INE173A01025,Standard Industries Ltd.,Standard Industries Ltd.,24.9,1000
7,INE457F01013,Salzer Electronics Ltd.,Salzer Electronics Ltd.,247.0,2000
8,INE736A01011,,,1254.65,125
9,INE335Y01020,Indian Railway Catering and Tourism Corporatio...,Indian Railway Catering and Tourism Corporatio...,713.55,575


In [11]:
process_dict

{'txn_id': '00989e54c0754fef995d92477a4ea51d',
 'consent_initiate_time': 1706868417,
 'consent_validate_time': 1706868504,
 'report_fetch_time': 1706868507,
 'report_receive_time': 1706868507}